# Day 13 - Prime number factors

* https://adventofcode.com/2020/day/13

For part 1, we need to find the next multiple of a bus ID that's equal to or greater than our earliest departure time. The bus  IDs, which determine their frequency, are all prime numbers, of course.

We can calculate the next bus departure $t$ for a given ID $b$ on or after earliest departure time $T$ as $t = b * \lceil T / b \rceil$ ($b$ multiplied by the ceiling of the division of $T$ by $b$).



In [1]:
import math

def parse_bus_ids(line: str) -> list[int]:
    return [int(b) for b in line.split(",") if b[0] != "x"]

def parse_input(lines: str) -> [int, list[int]]:
    return int(lines[0]), parse_bus_ids(lines[1])

def earliest_departure(earliest: int, bus_ids: list[int]) -> tuple[int, int]:
    t, bid = min((bid * math.ceil(earliest / bid), bid) for bid in bus_ids)
    return t - earliest, bid

test_earliest, test_bus_ids = parse_input(["939", "7,13,x,x,59,x,31,19"])
assert earliest_departure(test_earliest, test_bus_ids) == (5, 59)

In [2]:
import aocd
data = aocd.get_data(day=13, year=2020).splitlines()
earliest, bus_ids = parse_input(data)

In [3]:
wait_time, bus_id = earliest_departure(earliest, bus_ids)
print("Part 1:", wait_time * bus_id)

Part 1: 3035


## Part 2: Chinese remainder theorem.

For part 2, we need to use the [Chinese remainder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem); this theorem was first introduced by the Chinese mathematician Sun-tzu (quote from the Wikipedia article):

> There are certain things whose number is unknown. If we count them by threes, we have two left over; by fives, we have three left over; and by sevens, two are left over. How many things are there?

We need to find a number that if counted in prime number steps, have an offset left over, where the offset is the prime number minus the index in the bus ids list, modulo the bus id (the matching time stamp lies X minutes *before* the next bus departs).

I adapted the [Rossetta Stone Python implementation](https://rosettacode.org/wiki/Chinese_remainder_theorem#Python) for this:

In [4]:
from functools import reduce
from operator import mul
from typing import Optional

def solve_chinese_remainder(bus_times: list[Optional[int]]) -> int:
    product = reduce(mul, (bid for bid in filter(None, bus_times)))
    summed = sum(
        ((bid - i) % bid) * mul_inv((factor := product // bid), bid) * factor
        for i, bid in enumerate(bus_times)
        if bid is not None
    )
    return summed % product

def mul_inv(a: int, b: int) -> int:
    if b == 1: return 1
    b0, x0, x1 = b, 0, 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += b0
    return x1

def parse_bus_times(line: str) -> list[Optional[int]]:
    return [None if bus_id == "x" else int(bus_id) for bus_id in line.split(",")]

tests = {
    "7,13,x,x,59,x,31,19": 1068781,
    "17,x,13,19": 3417,
    "67,7,59,61": 754018,
    "67,x,7,59,61": 779210,
    "67,7,x,59,61": 1261476,
    "1789,37,47,1889": 1202161486,
}
for times, expected in tests.items():
    assert solve_chinese_remainder(parse_bus_times(times)) == expected

In [5]:
print("Part 2:", solve_chinese_remainder(parse_bus_times(data[1])))

Part 2: 725169163285238
